In [2]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import optuna

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
def remove_stopwords(df):
    article_ls = ['a','an','the']
    aux_verb = ['have','has','had','having','be','been','do','does','did','doing','am','is','are','was','were','will','should','can']
    conjunctions = ['and','nor','but','or','so','as','both','if','because','as','when','while']
    prepositions = ['about','of','at','by','to','in','on','from','with']
    others = ['what','which','who','whom','this','that','these','those']
    nationalities = ['czech', 'austrian', 'norwegian', 'swedish', 'eur', 'danish', 'finnish', 'afghan', 'iranian', 'nigerian', 'african', 'french', 'south']

    remove_ls = []
    [remove_ls.append(i) for i in article_ls if i in df.index]
    [remove_ls.append(i) for i in aux_verb if i in df.index]
    [remove_ls.append(i) for i in conjunctions if i in df.index]
    [remove_ls.append(i) for i in prepositions if i in df.index]
    [remove_ls.append(i) for i in others if i in df.index]
    [remove_ls.append(i) for i in nationalities if i in df.index]

    df = df.drop(remove_ls)

    return(df)

In [6]:
top_words = 1000

In [29]:
df_peace

Unnamed: 0,year,price,for,share,stock,average,total,section,month,up,...,response,away,wide,defer,personal,career,roe,en,conduct,y_target
Australia,0.020267,0.012453,0.017356,0.009052,0.007882,0.006641,0.007970,0.007963,0.006885,0.006907,...,0.000181,0.000251,0.000198,0.000068,0.000202,0.000297,0.000076,0.000022,0.000171,1
New Zealand,0.011019,0.004944,0.018757,0.003139,0.002607,0.002598,0.003010,0.002332,0.003514,0.006357,...,0.000292,0.000684,0.000253,0.000041,0.000224,0.000328,0.000034,0.000022,0.000132,1
Sweden,0.028005,0.030082,0.013544,0.014041,0.016080,0.015304,0.012021,0.014981,0.014903,0.011312,...,0.000018,0.000007,0.000031,0.000161,0.000073,0.000013,0.000168,0.000015,0.000060,1
Austria,0.027920,0.028417,0.012185,0.012490,0.016386,0.015253,0.015992,0.013357,0.013980,0.014063,...,0.000023,0.000006,0.000111,0.000219,0.000058,0.000091,0.000234,0.000007,0.000024,1
Belgium,0.014377,0.009858,0.020096,0.005466,0.005843,0.005699,0.005709,0.005499,0.005363,0.006247,...,0.000477,0.000155,0.000331,0.000100,0.000308,0.000252,0.000083,0.000992,0.000247,1
Denmark,0.024203,0.023034,0.015917,0.016335,0.013346,0.011969,0.011763,0.011392,0.011463,0.010751,...,0.000077,0.000011,0.000052,0.000157,0.000096,0.000037,0.000157,0.000079,0.000108,1
Norway,0.024615,0.024702,0.014148,0.019393,0.014107,0.012711,0.013142,0.012909,0.011239,0.012091,...,0.000033,0.000004,0.000079,0.000201,0.000022,0.000030,0.000125,0.000012,0.000057,1
Finland,0.027506,0.025023,0.013652,0.014648,0.014547,0.013763,0.014425,0.012225,0.011570,0.011662,...,0.000030,0.000007,0.000070,0.000215,0.000031,0.000053,0.000187,0.000024,0.000037,1
Netherlands,0.026058,0.022027,0.014867,0.013188,0.012551,0.013118,0.013476,0.012303,0.010902,0.010403,...,0.000097,0.000068,0.000112,0.000103,0.000139,0.000072,0.000189,0.000103,0.000171,1
Czech Republic,0.017310,0.010163,0.016939,0.005099,0.005184,0.005613,0.004711,0.005522,0.005431,0.004744,...,0.000063,0.000098,0.000051,0.000024,0.000135,0.000111,0.000029,0.000005,0.000273,1


In [7]:
# Read the tab-delimited file into a DataFrame
df_peace = pd.read_csv('data/peace_10K.txt', delimiter='\t')
df_peace = df_peace.drop('Unnamed: 1',axis=1)
df_peace = df_peace.set_index('Unnamed: 0')
df_peace = remove_stopwords(df_peace)
df_peace = df_peace/df_peace.sum()
df_peace = df_peace.transpose()

sum1 = df_peace.sum()
sorted_series = sum1.sort_values(ascending=False)
# Display the sorted indices
top_indices = sorted_series.index.tolist()[:top_words]
df_peace = df_peace[top_indices]
df_peace['y_target'] = 1

In [8]:
# Read the tab-delimited file into a DataFrame
df_nonpeace = pd.read_csv('data/nopeace_10K.txt', delimiter='\t')
df_nonpeace = df_nonpeace.drop('Unnamed: 1',axis=1)
df_nonpeace = df_nonpeace.set_index('Unnamed: 0')
df_nonpeace = remove_stopwords(df_nonpeace)
df_nonpeace = df_nonpeace/df_nonpeace.sum()
df_nonpeace = df_nonpeace.transpose()

sum1 = df_nonpeace.sum()
sorted_series = sum1.sort_values(ascending=False)
# Display the sorted indices
top_indices = sorted_series.index.tolist()[:top_words]
df_nonpeace = df_nonpeace[top_indices]

df_nonpeace['y_target'] = 0

In [9]:
country_list = list(df_peace.index.values) + list(df_nonpeace.index.values)

In [10]:
def get_result(df_peace,df_nonpeace,val,model):

    df_X_peace = df_peace.copy(deep=True)
    df_X_nonpeace = df_nonpeace.copy(deep=True)

    if val in df_peace.index.values:
        y = df_peace.loc[val]
        df_X_peace = df_X_peace.drop([val])

    else:
        y = df_nonpeace.loc[val]
        df_X_nonpeace = df_X_nonpeace.drop([val])

    df_X_peace = df_X_peace.mean()
    df_X_nonpeace = df_X_nonpeace.mean()


    df = pd.concat([df_X_peace,df_X_nonpeace],axis=1)
    df = df.transpose()
    df = df.fillna(0)

    y = pd.DataFrame(y).transpose()
    df = pd.concat([df,y]).fillna(0)

    test = df.iloc[-1:]
    train = df.drop(val)

    X_train = train.drop("y_target", axis=1)
    y_train = train["y_target"]

    X_test = test.drop("y_target", axis=1)
    y_test = test["y_target"]

    X_train = np.log(X_train)

    X_test = np.log(X_test)

    X_train[X_train < -1000000] = -99

    X_test[X_test < -1000000] = -99

    model.fit(X_train, y_train)

    return(model.predict(X_test),y_test)

SVM

In [10]:
def objective(trial):
    
    param = {
        'C': trial.suggest_loguniform('C', 1e-10, 1e10),
        'kernel': trial.suggest_categorical('kernel', ['linear']),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        'degree': trial.suggest_int('degree', 1, 5)
    }

    clf = SVC(**param,random_state=1)
    
    pred_ls = []
    y_ls = []

    for i in country_list:
        pred,y = get_result(df_peace,df_nonpeace,i,clf)
        pred_ls.append(pred)
        y_ls.append(y)

    return(accuracy_score(pred_ls,y_ls))

In [11]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)
trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

[I 2024-10-16 18:17:36,419] A new study created in memory with name: no-name-7fbc3d6a-aa72-416e-b28e-4728acb3dbf6
[I 2024-10-16 18:17:37,143] Trial 0 finished with value: 1.0 and parameters: {'C': 1.2094491856062256e-09, 'kernel': 'linear', 'gamma': 'auto', 'degree': 1}. Best is trial 0 with value: 1.0.
[I 2024-10-16 18:17:37,836] Trial 1 finished with value: 1.0 and parameters: {'C': 60413489.83848079, 'kernel': 'linear', 'gamma': 'scale', 'degree': 5}. Best is trial 0 with value: 1.0.
[I 2024-10-16 18:17:38,545] Trial 2 finished with value: 1.0 and parameters: {'C': 10.314749791107397, 'kernel': 'linear', 'gamma': 'scale', 'degree': 5}. Best is trial 0 with value: 1.0.
[I 2024-10-16 18:17:39,246] Trial 3 finished with value: 1.0 and parameters: {'C': 773.1426983699209, 'kernel': 'linear', 'gamma': 'scale', 'degree': 1}. Best is trial 0 with value: 1.0.
[I 2024-10-16 18:17:39,933] Trial 4 finished with value: 1.0 and parameters: {'C': 0.0014911997404844061, 'kernel': 'linear', 'gamma'

Best Score:  1.0
Best Params: 
  C: 1.2094491856062256e-09
  kernel: linear
  gamma: auto
  degree: 1


In [9]:
param = {
        'C': 1.2094491856062256e-09,
        'kernel': 'linear',
        'gamma': 'auto',
        'degree': 1
    }


clf = SVC(**param)

pred_ls = []
y_ls = []

for i in country_list:
    pred,y = get_result(df_peace,df_nonpeace,i,clf)
    pred_ls.append(pred)
    y_ls.append(y)

print(accuracy_score(pred_ls,y_ls))

1.0


RF

In [11]:
#from sklearn.model_selection import cross_val_score
def objective(trial):

    n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
    max_depth = trial.suggest_int("max_depth", 1, top_words)

    rf_model = RandomForestClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    random_state=1,
    )
    
    pred_ls = []
    y_ls = []

    for i in country_list:
        pred,y = get_result(df_peace,df_nonpeace,i,rf_model)
        pred_ls.append(pred)
        y_ls.append(y)

    #print(accuracy_score(pred_ls,y_ls))


    return(accuracy_score(pred_ls,y_ls))

In [12]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)
trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

[I 2024-11-03 22:59:11,780] A new study created in memory with name: no-name-b783f3a1-8059-4e53-a006-99bed99b7a45
[I 2024-11-03 22:59:13,825] Trial 0 finished with value: 0.9 and parameters: {'n_estimators': 93, 'max_depth': 703}. Best is trial 0 with value: 0.9.
[I 2024-11-03 22:59:14,618] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 1, 'max_depth': 775}. Best is trial 0 with value: 0.9.
[I 2024-11-03 22:59:15,566] Trial 2 finished with value: 0.75 and parameters: {'n_estimators': 10, 'max_depth': 191}. Best is trial 0 with value: 0.9.
[I 2024-11-03 22:59:16,734] Trial 3 finished with value: 0.8 and parameters: {'n_estimators': 29, 'max_depth': 175}. Best is trial 0 with value: 0.9.
[I 2024-11-03 22:59:17,749] Trial 4 finished with value: 0.75 and parameters: {'n_estimators': 22, 'max_depth': 853}. Best is trial 0 with value: 0.9.
[I 2024-11-03 22:59:18,742] Trial 5 finished with value: 0.8 and parameters: {'n_estimators': 17, 'max_depth': 659}. Best is trial 0 wi

Best Score:  0.95
Best Params: 
  n_estimators: 198
  max_depth: 584


## Testing on Twitter

In [20]:
df_tweet = pd.read_csv('Tweets.csv')

In [21]:
df_tweet = df_tweet[['text','country_code']]

In [22]:
import re
import pycountry

country_names = [country.name for country in pycountry.countries]

# Function to remove countries from text
def remove_countries_from_text(text):
    pattern = r'\b(?:' + '|'.join(re.escape(country) for country in country_names) + r')\b'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return ' '.join(cleaned_text.split())


def preprocess_tweet_v1(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    # Expand contractions (basic example)
    contractions = {"can't": "cannot", "I'm": "I am"}  # Expand as needed
    for contraction, expansion in contractions.items():
        tweet = tweet.replace(contraction, expansion)

    cleaned_tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
        # Clean up multiple spaces
    cleaned_tweet = re.sub(r'\s+', ' ', cleaned_tweet).strip()

    cleaned_tweet =  remove_countries_from_text(cleaned_tweet)

    cleaned_tweet = cleaned_tweet.lower()

    return(cleaned_tweet)


In [23]:
df_tweet = df_tweet[~df_tweet['country_code'].isnull()]

In [24]:
df_tweet['text'] = df_tweet['text'].apply(lambda x: preprocess_tweet_v1(x))

In [27]:
article_ls = ['a', 'an', 'the']
aux_verb = ['have', 'has', 'had', 'having', 'be', 'been', 'do', 'does', 'did', 'doing', 'am', 'is', 'are', 'was', 'were', 'will', 'should', 'can']
conjunctions = ['and', 'nor', 'but', 'or', 'so', 'as', 'both', 'if', 'because', 'when', 'while']
prepositions = ['about', 'of', 'at', 'by', 'to', 'in', 'on', 'from', 'with']
others = ['what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those']
nationalities = ['czech', 'austrian', 'norwegian', 'swedish', 'eur', 'danish', 'finnish', 'afghan', 'iranian', 'nigerian', 'african', 'french', 'south']

# Combine all stopwords into a set
stopwords = set(article_ls + aux_verb + conjunctions + prepositions + others + nationalities)

# Function to remove stopwords
def remove_stopwords_v2(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

# Apply the function to the 'text' column
df_tweet['text'] = df_tweet['text'].apply(lambda x: remove_stopwords_v2(x))


In [87]:
xt.columns

Index(['year', 'price', 'for', 'share', 'stock', 'average', 'total', 'section',
       'month', 'up',
       ...
       'channel', 'arrive', 'objective', 'text', 'illegal', 'culture',
       'mittee', 'dispute', 'agriculture', 'resolve'],
      dtype='object', name='Unnamed: 0', length=1269)

In [52]:
def filter_words(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word in xt.columns]  # Retain only allowed words
    return ' '.join(filtered_words)  # Join the filtered words back into a single string

# Apply the function to the DataFrame
df_tweet['filtered_text'] = df_tweet['text'].apply(filter_words)

In [89]:
df_tweet

,text,country_code,filtered_text
0,hi,LK,hi
1,karapitiya childrens cancer ward closed,LK,
2,somewhere mullaitivu district srilanka lka sat...,LK,district
3,good morning friends my way,LK,good morning my way
4,just posted photo ceylon cinnamon powder,LK,just photo
...,...,...,...
2192864,courtois saves usual,NO,
2192865,vinicius playing like he today,NO,like he today
2192866,my freedom day,NO,my day
2192867,i want rodrygo ga,NO,want


In [90]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_tweet['filtered_text'])

# Create a DataFrame from the sparse matrix
words_df = pd.DataFrame.sparse.from_spmatrix(X, columns=vectorizer.get_feature_names_out())



In [92]:
words_df['country_code'] = df_tweet['country_code']

# Group by country_code and sum occurrences
pivot_df = words_df.groupby('country_code').sum()

# Display the pivot DataFrame
print(pivot_df)

              able  above  abstract  access  accord  accordance  account  \
country_code                                                               
CA             222     86        26     112       1           7      198   
FI             358    272        11      99       2          16      347   
GB             267    175        22     201       1           1      242   
IE             293    106        17     150       2           2      203   
IN             319    372        13     262       6         148      355   
IR             216     77        18     161       2           1      176   
LK             258    126        19      79      12           5      201   
NG             317    148        12     180      95           0      393   
UG              40     12         6      15       0           0       18   

              accuse  achieve  acquire  ...  write  year  yesterday  yet  \
country_code                            ...                                
CA         

In [93]:
pivot_df

,able,above,abstract,access,accord,accordance,account,accuse,achieve,acquire,...,write,year,yesterday,yet,yield,you,young,your,youth,yr
country_code,,,,,,,,,,,,,,,,,,,,,
CA,222,86,26,112,1,7,198,2,89,4,...,100,987,262,390,9,14933,237,5107,89,28
FI,358,272,11,99,2,16,347,5,107,22,...,99,5225,354,423,11,14599,314,5891,192,112
GB,267,175,22,201,1,1,242,4,85,7,...,133,1466,584,509,11,17789,415,6710,268,33
IE,293,106,17,150,2,2,203,1,37,5,...,123,1791,676,502,12,11868,383,3901,167,44
IN,319,372,13,262,6,148,355,5,128,21,...,90,990,359,850,16,24354,544,11012,398,44
IR,216,77,18,161,2,1,176,1,53,0,...,90,1609,527,387,6,11350,425,4173,161,53
LK,258,126,19,79,12,5,201,7,120,8,...,115,1159,552,451,10,16135,321,6076,308,26
NG,317,148,12,180,95,0,393,8,101,11,...,142,1403,439,492,19,19562,342,8406,250,48
UG,40,12,6,15,0,0,18,0,3,0,...,28,141,74,50,1,1370,41,392,8,3


In [94]:
country_codes = ['CA', 'FI', 'GB', 'IE', 'IN', 'IR', 'LK', 'NG', 'UG']

pivot_df_v2 = pd.DataFrame(pivot_df, index=country_codes)

# Dictionary to map country codes to country names
countries = {
    'IN': 'India',
    'IR': 'Iran',
    'NG': 'Nigeria',
    'UG': 'Uganda',
    'GM': 'Gambia',
    'LY': 'Libya',
    'PK': 'Pakistan',
    'ZW': 'Zimbabwe',
    'CA': 'Canada',
    'GB': 'United Kingdom',
    'FI': 'Finland',
    'NO': 'Norway',
    'IE': 'Ireland',
    'FR': 'France',
    'AU': 'Australia',
    'SG': 'Singapore'
}

# Replace the index with country names
pivot_df_v2.index = pivot_df_v2.index.map(countries)

# Display the updated DataFrame
print(pivot_df_v2)

                able  above  abstract  access  accord  accordance  account  \
Canada           222     86        26     112       1           7      198   
Finland          358    272        11      99       2          16      347   
United Kingdom   267    175        22     201       1           1      242   
Ireland          293    106        17     150       2           2      203   
India            319    372        13     262       6         148      355   
Iran             216     77        18     161       2           1      176   
NaN              258    126        19      79      12           5      201   
Nigeria          317    148        12     180      95           0      393   
Uganda            40     12         6      15       0           0       18   

                accuse  achieve  acquire  ...  write  year  yesterday  yet  \
Canada               2       89        4  ...    100   987        262  390   
Finland              5      107       22  ...     99  5225     

In [95]:
pivot_df_v2 = pivot_df_v2[pivot_df_v2.index.notna()]

In [96]:
pivot_df_v2.to_csv('tweet_clean.csv')

In [34]:
pivot_df_v2 = pd.read_csv('tweet_clean.csv',index_col=0)

In [35]:
pivot_df_v2

,able,above,abstract,access,accord,accordance,account,accuse,achieve,acquire,...,write,year,yesterday,yet,yield,you,young,your,youth,yr
Canada,222,86,26,112,1,7,198,2,89,4,...,100,987,262,390,9,14933,237,5107,89,28
Finland,358,272,11,99,2,16,347,5,107,22,...,99,5225,354,423,11,14599,314,5891,192,112
United Kingdom,267,175,22,201,1,1,242,4,85,7,...,133,1466,584,509,11,17789,415,6710,268,33
Ireland,293,106,17,150,2,2,203,1,37,5,...,123,1791,676,502,12,11868,383,3901,167,44
India,319,372,13,262,6,148,355,5,128,21,...,90,990,359,850,16,24354,544,11012,398,44
Iran,216,77,18,161,2,1,176,1,53,0,...,90,1609,527,387,6,11350,425,4173,161,53
Nigeria,317,148,12,180,95,0,393,8,101,11,...,142,1403,439,492,19,19562,342,8406,250,48
Uganda,40,12,6,15,0,0,18,0,3,0,...,28,141,74,50,1,1370,41,392,8,3


In [36]:
pivot_df_v2 = pivot_df_v2.transpose()

In [37]:
pivot_df_v2 = pivot_df_v2/pivot_df_v2.sum()
pivot_df_v2 = pivot_df_v2.transpose()

# sum1 = pivot_df_v2.sum()
# sorted_series = sum1.sort_values(ascending=False)
# # Display the sorted indices
# top_indices = sorted_series.index.tolist()[:top_words]
# pivot_df_v2 = pivot_df_v2[top_indices]

In [17]:
df_nonpeace

Unnamed: 0,for,say,year,he,it,price,not,we,they,their,...,really,sit,coronavirus,highly,mittee,dispute,exercise,agriculture,resolve,y_target
India,0.021258,0.005261,0.013650,0.003909,0.006556,0.009879,0.004355,0.004032,0.003081,0.002583,...,0.000139,0.000099,0.000131,0.000095,0.000101,0.000069,0.000165,0.000081,0.000073,0
Iran,0.014953,0.008767,0.019020,0.005159,0.006553,0.025410,0.003367,0.002655,0.002611,0.002306,...,0.000090,0.000078,0.000352,0.000104,0.000053,0.000101,0.000083,0.000082,0.000150,0
Nigeria,0.017916,0.011285,0.009525,0.013195,0.010488,0.005841,0.009358,0.010065,0.007852,0.005455,...,0.000232,0.000254,0.000152,0.000118,0.000317,0.000089,0.000385,0.000204,0.000289,0
Kenya,0.016901,0.012312,0.009947,0.011233,0.010461,0.006764,0.008567,0.007479,0.008189,0.005660,...,0.000354,0.000297,0.000204,0.000109,0.000232,0.000193,0.000203,0.000154,0.000161,0
Congo,0.019906,0.014075,0.005046,0.013275,0.010823,0.000543,0.009822,0.011069,0.009598,0.006914,...,0.000261,0.000265,0.000000,0.000149,0.000352,0.000150,0.000341,0.000403,0.000291,0
Zimbabwe,0.019387,0.014608,0.005248,0.012969,0.011920,0.000841,0.010670,0.015040,0.010748,0.007052,...,0.000375,0.000273,0.000236,0.000163,0.000270,0.000144,0.000279,0.000456,0.000211,0
Sri Lanka,0.013336,0.003307,0.022906,0.004226,0.005449,0.019907,0.002738,0.003244,0.002597,0.002182,...,0.000084,0.000071,0.000122,0.000103,0.000104,0.000034,0.000065,0.000095,0.000059,0
Uganda,0.018789,0.014875,0.004941,0.014489,0.011720,0.000639,0.011875,0.009920,0.012560,0.006528,...,0.000290,0.000490,0.000173,0.000142,0.000289,0.000159,0.000332,0.000223,0.000219,0
Afghanistan,0.017550,0.026479,0.004160,0.011940,0.005803,0.000377,0.007797,0.005279,0.007887,0.007106,...,0.000112,0.000107,0.000558,0.000077,0.000206,0.000120,0.000069,0.000223,0.000467,0
Guinea,0.028448,0.002586,0.002586,0.003448,0.010345,0.002586,0.002586,0.000862,0.001724,0.003448,...,0.000000,0.000000,0.000000,0.000862,0.000000,0.000862,0.000000,0.000000,0.000000,0


In [39]:
def get_result_final(df_peace,df_nonpeace,model):

    df_X_peace = df_peace.copy(deep=True)
    df_X_nonpeace = df_nonpeace.copy(deep=True)

    df_X_peace = df_X_peace.mean()
    df_X_nonpeace = df_X_nonpeace.mean()


    df = pd.concat([df_X_peace,df_X_nonpeace],axis=1)
    df = df.transpose()
    df = df.fillna(0)

    X_train = df.drop("y_target", axis=1)
    y_train = df["y_target"]


    X_train = np.log(X_train)


    X_train[X_train < -1000000] = -99


    model.fit(X_train, y_train)

    return(X_train,model)

In [101]:
param = {
        'C': 1.2094491856062256e-09,
        'kernel': 'linear',
        'gamma': 'auto',
        'degree': 1
    }


clf = SVC(**param)

xt,mod = get_result_final(df_peace,df_nonpeace,clf)

In [40]:
n_estimators = 198
max_depth = 584

rf_model = RandomForestClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    random_state=1
    )

xt,mod = get_result_final(df_peace,df_nonpeace,rf_model)

In [41]:
xt.columns

Index(['year', 'price', 'for', 'share', 'stock', 'average', 'total', 'section',
       'month', 'up',
       ...
       'channel', 'arrive', 'objective', 'text', 'illegal', 'culture',
       'mittee', 'dispute', 'agriculture', 'resolve'],
      dtype='object', name='Unnamed: 0', length=1269)

In [42]:
pivot_df_v3 = pivot_df_v2.reindex(columns=xt.columns, fill_value=0)
pivot_df_v3 = np.log(pivot_df_v3)
pivot_df_v3[pivot_df_v3 < -1000000] = -99

In [44]:
pivot_df_v3

Unnamed: 0,year,price,for,share,stock,average,total,section,month,up,...,channel,arrive,objective,text,illegal,culture,mittee,dispute,agriculture,resolve
Canada,-5.911634,-7.167949,-2.739762,-6.357414,-8.914483,-9.168718,-8.375487,-9.222785,-7.454446,-5.043708,...,-7.946491,-9.587428,-12.113157,-8.502239,-9.548207,-8.096774,-99.0,-10.860394,-9.861865,-10.033715
Finland,-4.478713,-7.349563,-3.235096,-6.868222,-8.645473,-8.996871,-7.841426,-8.540113,-6.843479,-5.326138,...,-5.922717,-9.302253,-9.861869,-9.168722,-8.330392,-7.373496,-99.0,-11.653628,-8.881040,-8.865535
United Kingdom,-5.687509,-7.267375,-3.176237,-7.056224,-8.043328,-9.007510,-8.198679,-9.127655,-6.949524,-4.775320,...,-7.871857,-9.085982,-10.412853,-8.352829,-9.481295,-7.973856,-99.0,-11.879190,-9.314241,-10.579907
Ireland,-5.491578,-7.709108,-3.150170,-7.144377,-8.638302,-9.050282,-8.337717,-8.887763,-7.210667,-4.821589,...,-8.194616,-9.110907,-10.679523,-8.516200,-8.600081,-7.176973,-99.0,-11.372670,-9.175445,-10.679523
India,-6.218190,-7.026850,-3.208415,-6.486531,-8.111948,-9.184069,-7.900959,-8.853215,-6.306855,-5.024573,...,-5.749449,-10.024852,-10.630988,-8.443066,-9.072843,-7.868870,-99.0,-12.017282,-8.772089,-9.309232
Iran,-5.561668,-7.921156,-3.078161,-7.023458,-8.563010,-8.937703,-8.490689,-8.627548,-6.961100,-4.777117,...,-8.490689,-9.361518,-11.335599,-8.682357,-8.360069,-7.934401,-99.0,-12.945037,-10.054665,-10.865595
Nigeria,-5.892618,-6.367051,-3.140280,-7.020889,-7.850719,-9.113635,-8.420488,-8.617198,-5.729244,-4.834492,...,-8.310673,-9.704999,-10.836401,-8.402788,-9.332324,-8.402788,-99.0,-12.040374,-9.131653,-10.366398
Uganda,-5.786266,-8.026976,-3.264232,-6.950837,-8.943267,-8.250120,-7.844654,-8.537802,-6.906385,-4.792227,...,-7.599532,-8.537802,-10.041879,-8.250120,-9.636414,-8.095969,-99.0,-10.735026,-10.735026,-10.735026


In [46]:
mod.predict(pivot_df_v3)

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [149]:
xt

Unnamed: 0,year,price,for,share,stock,average,total,section,month,up,...,channel,arrive,objective,text,illegal,culture,mittee,dispute,agriculture,resolve
0,-3.810912,-3.959626,-4.151162,-4.484281,-4.523290,-4.578826,-4.583221,-4.620441,-4.653836,-4.661338,...,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.000000
1,-4.635319,-4.922794,-3.971537,-5.396933,-5.623206,-5.795343,-5.837753,-5.694196,-5.420167,-5.458681,...,-8.537481,-8.537979,-8.542081,-8.545671,-8.546821,-8.548532,-8.5565,-8.556665,-8.557561,-8.558341


In [84]:
pivot_df_v3.shape

(8, 1269)

## News data

In [47]:
df_news = pd.read_csv('clean_data.csv')

In [48]:
df_news = df_news[['country','cleaned_text']]

In [49]:
df_news = df_news[df_news['cleaned_text'].notnull()]

In [53]:
df_news['filtered_text'] = df_news['cleaned_text'].apply(filter_words)

In [54]:
df_news

,country,cleaned_text,filtered_text
0,Australia,second video possible show missile launcher he...,possible show head government claim could may ...
1,Australia,member chinese passenger board flight burst pr...,member board press help claim authority relati...
2,Australia,year secrecy english box promoter family value...,year family value family life anti anti family
3,Australia,carry strike attempt stop extremist attack min...,carry strike attempt stop attack group little ...
4,Australia,dramatically raise stake conflict call first t...,raise conflict call time consider east former ...
...,...,...,...
618557,Zimbabwe,today preside burial two national hero nationa...,today national national national also expect t...
618558,Zimbabwe,massive rebranding exercise set launch new pre...,exercise set launch new voice home away servic...
618559,Zimbabwe,authority student still come term mysterious f...,authority student still come term fire school ...
618560,Zimbabwe,country detractor local stooge nothing new wha...,country local nothing new power cause country ...


In [56]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_news['filtered_text'])

# Create a DataFrame from the sparse matrix
words_df = pd.DataFrame.sparse.from_spmatrix(X, columns=vectorizer.get_feature_names_out())

In [57]:
words_df

,able,abstract,access,accord,accordance,account,accuse,achieve,acquire,acquisition,...,worth,would,wound,write,year,yesterday,yet,yield,young,youth
0,0,0,1,0,0,1,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
617435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
617436,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
617437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
words_df['country_code'] = df_news['country']

In [59]:
words_df

,able,abstract,access,accord,accordance,account,accuse,achieve,acquire,acquisition,...,would,wound,write,year,yesterday,yet,yield,young,youth,country_code
0,0,0,1,0,0,1,0,0,0,0,...,2,0,0,1,0,0,0,0,0,Australia
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Australia
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Australia
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Australia
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Zimbabwe
617435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zimbabwe
617436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zimbabwe
617437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Zimbabwe


In [60]:
# Group by country_code and sum occurrences
pivot_df = words_df.groupby('country_code').sum()

# Display the pivot DataFrame
print(pivot_df)

              able  abstract  access  accord  accordance  account  accuse  \
country_code                                                                
Australia     1201         4     704    1769          43      778    1834   
Canada        3069        50    2062    6632          64     2161    2273   
Finland       1394         4    1000   12516         296     1539     888   
France        1406         9    1339    5737         112     1434    4259   
Gambia        2763        23    2631   12988         406     1685   20944   
India         1937        19     946    3308         165     2060    7261   
Iran          2113        47    1503   14234         496     2047    1030   
Ireland       1121         4     368     990          32      780    1791   
Libya          336         0     286    4891         365      672    1042   
Nigeria       1825        17    1801    6977         318     2177    1813   
Norway         998         5     462    1553          95      502     477   

In [61]:
pivot_df

,able,abstract,access,accord,accordance,account,accuse,achieve,acquire,acquisition,...,worth,would,wound,write,year,yesterday,yet,yield,young,youth
country_code,,,,,,,,,,,,,,,,,,,,,
Australia,1201,4,704,1769,43,778,1834,277,91,27,...,419,8404,556,1681,10520,1709,1108,34,1906,488
Canada,3069,50,2062,6632,64,2161,2273,605,582,285,...,1028,19166,734,4435,30905,297,3279,308,2981,795
Finland,1394,4,1000,12516,296,1539,888,915,219,119,...,366,8205,975,900,19569,93,1178,126,1322,433
France,1406,9,1339,5737,112,1434,4259,564,126,46,...,536,14479,1629,1834,20355,382,1989,130,2075,596
Gambia,2763,23,2631,12988,406,1685,20944,2871,612,158,...,839,18851,78,2647,18684,6059,2213,359,5926,7189
India,1937,19,946,3308,165,2060,7261,766,634,448,...,1688,14435,41,2561,19050,129,2662,276,924,1716
Iran,2113,47,1503,14234,496,2047,1030,2449,314,33,...,2478,10722,408,3866,33445,199,1862,303,2089,729
Ireland,1121,4,368,990,32,780,1791,303,66,22,...,664,6644,90,1391,10350,1294,1220,30,1624,199
Libya,336,0,286,4891,365,672,1042,1280,34,17,...,366,3624,559,277,2940,385,545,19,321,334


In [62]:
pivot_df_v2 = pivot_df.transpose()

In [63]:
pivot_df_v2

country_code,Australia,Canada,Finland,France,Gambia,India,Iran,Ireland,Libya,Nigeria,Norway,Pakistan,Singapore,UK,Uganda,Zimbabwe
able,1201,3069,1394,1406,2763,1937,2113,1121,336,1825,998,593,2798,3723,1477,2039
abstract,4,50,4,9,23,19,47,4,0,17,5,10,22,50,6,15
access,704,2062,1000,1339,2631,946,1503,368,286,1801,462,453,1988,1841,3226,2138
accord,1769,6632,12516,5737,12988,3308,14234,990,4891,6977,1553,2200,6146,5598,5028,3067
accordance,43,64,296,112,406,165,496,32,365,318,95,174,140,68,193,231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yesterday,1709,297,93,382,6059,129,199,1294,385,988,43,241,6400,1900,470,9423
yet,1108,3279,1178,1989,2213,2662,1862,1220,545,2406,679,978,2335,4275,3737,4022
yield,34,308,126,130,359,276,303,30,19,333,129,164,516,225,257,276
young,1906,2981,1322,2075,5926,924,2089,1624,321,1941,1329,633,2498,3874,1084,2216


In [64]:
pivot_df_v2 = pivot_df_v2/pivot_df_v2.sum()
pivot_df_v2 = pivot_df_v2.transpose()

In [65]:
pivot_df_v2

,able,abstract,access,accord,accordance,account,accuse,achieve,acquire,acquisition,...,worth,would,wound,write,year,yesterday,yet,yield,young,youth
country_code,,,,,,,,,,,,,,,,,,,,,
Australia,0.001097,0.000004,0.000643,0.001616,0.000039,0.000711,0.001676,0.000253,0.000083,0.000025,...,0.000383,0.007679,0.000508,0.001536,0.009612,0.001561,0.001012,0.000031,0.001741,0.000446
Canada,0.001120,0.000018,0.000753,0.002420,0.000023,0.000789,0.000830,0.000221,0.000212,0.000104,...,0.000375,0.006995,0.000268,0.001619,0.011279,0.000108,0.001197,0.000112,0.001088,0.000290
Finland,0.000853,0.000002,0.000612,0.007659,0.000181,0.000942,0.000543,0.000560,0.000134,0.000073,...,0.000224,0.005021,0.000597,0.000551,0.011975,0.000057,0.000721,0.000077,0.000809,0.000265
France,0.000665,0.000004,0.000633,0.002714,0.000053,0.000678,0.002015,0.000267,0.000060,0.000022,...,0.000254,0.006849,0.000771,0.000868,0.009629,0.000181,0.000941,0.000061,0.000982,0.000282
Gambia,0.001089,0.000009,0.001037,0.005119,0.000160,0.000664,0.008254,0.001131,0.000241,0.000062,...,0.000331,0.007429,0.000031,0.001043,0.007363,0.002388,0.000872,0.000141,0.002335,0.002833
India,0.000848,0.000008,0.000414,0.001449,0.000072,0.000902,0.003180,0.000335,0.000278,0.000196,...,0.000739,0.006321,0.000018,0.001121,0.008342,0.000056,0.001166,0.000121,0.000405,0.000751
Iran,0.000772,0.000017,0.000549,0.005199,0.000181,0.000748,0.000376,0.000894,0.000115,0.000012,...,0.000905,0.003916,0.000149,0.001412,0.012215,0.000073,0.000680,0.000111,0.000763,0.000266
Ireland,0.001204,0.000004,0.000395,0.001063,0.000034,0.000837,0.001923,0.000325,0.000071,0.000024,...,0.000713,0.007133,0.000097,0.001493,0.011112,0.001389,0.001310,0.000032,0.001744,0.000214
Libya,0.000431,0.000000,0.000367,0.006278,0.000469,0.000863,0.001337,0.001643,0.000044,0.000022,...,0.000470,0.004652,0.000718,0.000356,0.003774,0.000494,0.000700,0.000024,0.000412,0.000429


In [66]:
pivot_df_v3 = pivot_df_v2.reindex(columns=xt.columns, fill_value=0)
pivot_df_v3 = np.log(pivot_df_v3)
pivot_df_v3[pivot_df_v3 < -1000000] = -99

In [85]:
high_peace_countries = ['Canada', 'United Kingdom', 'Norway', 'Ireland', 'Finland', 
                        'Singapore', 'Australia', 'France']
low_peace_countries = ['India', 'Uganda', 'Iran', 'Nigeria', 
                       'Zimbabwe', 'Gambia', 'Libya', 'Pakistan']

# Divide the DataFrame
high_peace_df = pivot_df_v3[pivot_df_v3.index.isin(high_peace_countries)]
low_peace_df = pivot_df_v3[pivot_df_v3.index.isin(low_peace_countries)]

In [95]:
df_comb = pd.concat([high_peace_df.mean(),low_peace_df.mean()],axis=1)

In [99]:
df_peace = np.log(df_peace)
df_peace[df_peace < -1000000] = -99
df_peace.mean()

Unnamed: 0
year        -3.852576
price       -4.094362
for         -4.162577
share       -4.621441
stock       -4.662474
              ...    
career      -9.410034
roe         -9.158079
en         -10.416555
conduct     -9.227097
y_target     0.000000
Length: 1001, dtype: float64

In [100]:
df_nonpeace = np.log(df_nonpeace)
df_nonpeace[df_nonpeace < -1000000] = -99
df_nonpeace.mean()

Unnamed: 0
for            -3.991034
say            -4.686380
year           -4.858588
he             -4.809406
it             -4.750796
                 ...    
dispute        -8.941882
exercise      -17.686943
agriculture   -17.669519
resolve       -17.665878
y_target      -99.000000
Length: 1001, dtype: float64

In [97]:
df_comb.transpose()

Unnamed: 0,year,price,for,share,stock,average,total,section,month,up,...,channel,arrive,objective,text,illegal,culture,mittee,dispute,agriculture,resolve
0,-4.472783,-6.716989,-99.0,-6.606627,-8.168665,-7.599604,-6.905202,-8.660285,-5.640801,-10.470142,...,-8.512846,-7.007862,-9.453430,-8.657969,-7.784754,-8.179156,-99.000000,-8.026981,-9.718178,-8.449645
1,-4.886189,-6.944918,-99.0,-6.852417,-8.256105,-8.237335,-6.848364,-7.334714,-5.913671,-10.857102,...,-8.290809,-7.527851,-8.324026,-9.035927,-7.247789,-7.944413,-88.381688,-7.996026,-8.120131,-7.588777
